# Code on Correlating IDs vs. Wavelengths

In [124]:
# imports
import json

from scipy.optimize import curve_fit

sys.path.append(os.path.abspath("/Users/xavier/local/Python/PYPIT/src/"))
import armsgs as msgs
import ardebug
debug = ardebug.init()
last_updated = "26 November 2015"
version = '0.3'
verbose = False
msgs = msgs.get_logger((None, debug, last_updated, version, verbose))
import ararclines
import ararc as pyparc
import arutils

## Read in IDs

In [4]:
wvsoln_fil = '/Users/xavier/PYPIT/Tiffany/lris_red/MasterFrames/MasterWaveSoln_01.json'
with open(wvsoln_fil) as data_file:    
        wvsoln_dict = json.load(data_file)
wvsoln_dict.keys()

[u'function',
 u'ions',
 u'yrej',
 u'xnorm',
 u'fitc',
 u'nrej',
 u'mask',
 u'xrej',
 u'fmax',
 u'fmin',
 u'shift',
 u'xfit',
 u'yfit',
 u'spec']

## ID pixels

In [6]:
pix = np.array(wvsoln_dict['xfit'])*(wvsoln_dict['xnorm']-1)
pix

array([  140.93689967,   154.42571123,   194.07916869,   205.86298773,
         212.86313041,   253.05224277,   272.55370351,   307.26868399,
         335.39219804,   349.2515197 ,   378.94265744,   391.93324683,
         425.84238777,   456.89126564,   481.01486112,   499.6324469 ,
         530.18107916,   542.27304318,   607.60782398,   624.11784818,
         665.38725867,   714.8788845 ,   739.04367662,   871.12853433,
         893.42568193,   934.96578041,   956.46186979,  1005.91212302,
        1022.5280144 ,  1066.56388483,  1083.79186726,  1152.3883973 ,
        1186.31546381,  1217.15943528,  1233.12444534,  1246.16520617,
        1278.06325613,  1286.78211123,  1344.23944597,  1443.37024723,
        1501.14774051,  1537.09369339,  1651.17129411,  1677.04690861,
        1718.40233216,  1767.86758504,  1787.00940503,  1849.84686133,
        1857.67905523,  2018.04455387])

In [176]:
wvsoln_dict['xnorm']

2048.0

In [80]:
diff = np.abs(pix-1024)
asrt = np.argsort(diff)
inner9 = pix[asrt[0:9]]
inner9.sort()
x9 = inner9/2047.
inner9wv = np.array(wvsoln_dict['yfit'])[asrt[0:9]]
inner9wv.sort()
x9,inner9, inner9wv

(array([ 0.42556352,  0.43645612,  0.45674928,  0.46725055,  0.49140797,
         0.49952517,  0.52103756,  0.52945377,  0.56296453]),
 array([  871.12853433,   893.42568193,   934.96578041,   956.46186979,
         1005.91212302,  1022.5280144 ,  1066.56388483,  1083.79186726,
         1152.3883973 ]),
 array([ 6931.3787,  6967.352 ,  7034.352 ,  7069.167 ,  7149.012 ,
         7175.9154,  7247.1631,  7274.94  ,  7386.014 ]))

In [115]:
## Re-fit in pixel space (probably unnecessary)
func = 'polynomial'
mask, param9 = arutils.robust_polyfit(x9, inner9wv, 2, function=func)
wavefit = arutils.func_val(param9, x9, func)
rms = np.sqrt(np.mean((wavefit-inner9wv)**2))
param9, rms, mask

(array([ 5533.57607429,  3266.11890897,    43.32775718]),
 0.034446151915919911,
 array([0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [112]:
# Extrema
x9min = np.min(x9)
x9max = np.max(x9)

In [113]:
# Means
x9cen = np.mean([x9min,x9max])
#wv9cen = np.mean(inner9wv)
wv9cen = arutils.func_val(param9, x9cen, func)
dwv9cen = param9[1] + param9[2]*(x9max**2-x9min**2)/(x9max-x9min)
#
x9cen, wv9cen, dwv9cen

(0.49426402824610371, 7158.4860010091643, 3308.9496125644928)

### Fitting the whole lot

$\lambda - \lambda_9 - \Delta\lambda_9 (x-x_9) = \lambda_2 (x-x_9)^2 + \lambda_3 (x-x_9)^3$

In [118]:
x9val = x9 - x9cen
y9val = inner9wv - wv9cen - dwv9cen*x9val
yest = wv9cen + dwv9cen*x9val
xdb.xpcol(x9val, y9val) #np.array(wvsoln_dict['yfit']), yest)

-0.0687005039009  0.21920475682    
-0.0578079061497  0.149447648177   
-0.0375147461713  0.000403799833904
-0.0270134812088  0.0672471707628  
-0.00285605412898 -0.0234618056218 
0.00526113755873  0.0205599042415  
0.0267735315169   0.084832251157   
0.0351897417866   0.0129165397507  
0.0687005039009   0.201493224852   


In [120]:
#Setup
xval = np.array(wvsoln_dict['xfit'])-x9cen
yval = np.array(wvsoln_dict['yfit'])-wv9cen-dwv9cen*xval

In [123]:
def x2x3_fit(x, p0, p1):
    return p0*x**2 + p1*x**3

In [126]:
# Fit
#mask, param = arutils.robust_polyfit(xval, yval, 1, function=func)
#param
popt, pcov = curve_fit(x2x3_fit, xval, yval)
popt

array([  49.3506657 , -150.43874706])

In [131]:
yfit = wv9cen + dwv9cen*xval + popt[0]*xval**2 + popt[1]*xval**3
#xdb.xpcol(xval, yfit-np.array(wvsoln_dict['yfit']))
rms9 = np.sqrt(np.mean((yfit-np.array(wvsoln_dict['yfit']))**2))
rms9
# Excellent!

0.050942055471585132

### Working the other way..

$xx = (\lambda-\lambda_9)/\lambda_9$

$pix = p_0 + p_1 xx + p_2 xx^2 + p_3 xx^3$

Fix $p_0$ from $x_9$

Fix $p_1$ from $\Delta\lambda_9$

In [153]:
## Re-fit 
func = 'polynomial'
mask, pparam9 = arutils.robust_polyfit(inner9wv, inner9, 2, function=func)
pixfit = arutils.func_val(pparam9, inner9wv, func)
prms = np.sqrt(np.mean((pixfit-inner9)**2))
xdb.xpcol(inner9wv, inner9, pixfit)
pparam9, prms, mask

6931.3787 871.128534335 871.137716864
6967.352  893.425681931 893.428566434
7034.352  934.965780407 934.928217299
7069.167  956.461869785 956.483873863
7149.012  1005.91212302 1005.89737904
7175.9154 1022.5280144  1022.5399989 
7247.1631 1066.56388483 1066.5971758 
7274.94   1083.79186726 1083.76674168
7386.014  1152.3883973  1152.38648339


(array([ -3.54210588e+03,   6.53672138e-01,  -2.44787089e-06]),
 0.02132308346633198,
 array([0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [154]:
# Extrema
wv9min = np.min(inner9wv)
wv9max = np.max(inner9wv)

In [162]:
# Means
wv9cen = np.mean([wv9min,wv9max])
pix9cen = arutils.func_val(pparam9, wv9cen, func)
dpix9cen = pparam9[1]*wv9cen + pparam9[2]*(wv9max**2-wv9min**2)/(wv9max-wv9min)*wv9cen
#
wv9cen, pix9cen, dpix9cen

(7158.6963500000002, 1011.8885897264831, 4428.5485934417866)

In [164]:
# Checking
wv9val = (inner9wv - wv9cen)/wv9cen
inner9val = inner9 - pix9cen - dpix9cen*wv9val
#yest = wv9cen + dwv9cen*x9val
xdb.xpcol(wv9val, inner9, inner9val) #np.array(wvsoln_dict['yfit']), yest)

-0.0317540567285  871.128534335 -0.135672130857   
-0.0267289378743  893.425681931 -0.0925075678784  
-0.0173696918993  934.965780407 -0.000284690298642
-0.0125063762482  956.461869785 -0.0416249979578  
-0.00135280916057 1005.91212302 0.0145143965005   
0.00240533320009  1022.5280144  -0.0127102840032  
0.0123579414009   1066.56388483 -0.0524489004149  
0.0162381031848   1083.79186726 -0.00795148873378 
0.0317540567285   1152.3883973  -0.124575682561   


In [171]:
#Setup
xxval = (np.array(wvsoln_dict['yfit'])-wv9cen)/wv9cen
yyval = 2047*np.array(wvsoln_dict['xfit'])-pix9cen-dpix9cen*xxval

In [172]:
# Fit
ppopt, ppcov = curve_fit(x2x3_fit, xxval, yyval)
ppopt
## Will want to normalize to keep these numbers tractable..

array([-147.3954693 ,  970.41945234])

In [175]:
# Finally
yyfit = pix9cen + dpix9cen*xxval + ppopt[0]*xxval**2 + ppopt[1]*xxval**3
xdb.xpcol(xxval, yyfit, yyfit-np.array(wvsoln_dict['xfit'])*2047)
#rms9 = np.sqrt(np.mean((yfit-np.array(wvsoln_dict['yfit']))**2))
#rms9

-0.193818299054   140.952329623 0.0154299549505   
-0.190875584491   154.468104794 0.0423935628184   
-0.182237964319   194.070588876 -0.00857981560225 
-0.179663040185   205.856573718 -0.00641400848508 
-0.178128961986   212.874149015 0.0110186093297   
-0.169334651273   253.04349707  -0.00874570457773 
-0.165044973587   272.601063591 0.0473600842211   
-0.157435054498   307.239761103 -0.0289228826278  
-0.15123941526    335.389036139 -0.00316189742267 
-0.148189795758   349.228025402 -0.0234942998537  
-0.141636605386   378.929797567 -0.0128598685004  
-0.138767856804   391.916928117 -0.0163187129184  
-0.131266225589   425.835066993 -0.00732077519746 
-0.124389666283   456.874566343 -0.0166992932237  
-0.119038915514   480.993419294 -0.0214418284797  
-0.114897630768   499.641063556 0.00861665152763  
-0.108111912024   530.160695051 -0.0203841052549  
-0.105421198652   542.250634962 -0.0224082209515  
-0.0908504591063  607.608662514 0.000838530973738 
-0.0871679869478  624.098242022

### Inner fraction (e.g. half)

In [71]:
diff = np.abs(pix-1024)
innerhalfidx = diff < 512.
innerh = pix[innerhalfidx]
innerhwv = np.array(wvsoln_dict['yfit'])[innerhalfidx]
innerh, innerhwv

(array([  530.18107916,   542.27304318,   607.60782398,   624.11784818,
          665.38725867,   714.8788845 ,   739.04367662,   871.12853433,
          893.42568193,   934.96578041,   956.46186979,  1005.91212302,
         1022.5280144 ,  1066.56388483,  1083.79186726,  1152.3883973 ,
         1186.31546381,  1217.15943528,  1233.12444534,  1246.16520617,
         1278.06325613,  1286.78211123,  1344.23944597,  1443.37024723,
         1501.14774051]),
 array([ 6384.756 ,  6404.018 ,  6508.3255,  6534.6872,  6600.7754,
         6680.1205,  6718.8974,  6931.3787,  6967.352 ,  7034.352 ,
         7069.167 ,  7149.012 ,  7175.9154,  7247.1631,  7274.94  ,
         7386.014 ,  7440.9469,  7490.9335,  7516.721 ,  7537.8488,
         7589.5025,  7603.6384,  7696.6579,  7856.9844,  7950.362 ]))

In [72]:
## Re-fit in pixel space (probably unnecessary)
func = 'legendre'
mask, paramh = arutils.robust_polyfit(innerh, innerhwv, 2, function=func)
wavehfit = arutils.func_val(paramh, innerh, func)
rmsh = np.sqrt(np.mean((wavehfit-innerhwv)**2))
paramh, rmsh

(array([  7.16580069e+03,   7.83526066e+02,   1.88217983e+00]),
 0.35961639834686626)

## 2 parameters

In [27]:
## 2 param model
wcen = arutils.func_val(wvsoln_dict['fitc'], 0.5, wvsoln_dict['function'], minv=wvsoln_dict['fmin'], maxv=wvsoln_dict['fmax'])
ends = arutils.func_val(wvsoln_dict['fitc'], np.array([0.,1.]), wvsoln_dict['function'], minv=wvsoln_dict['fmin'], maxv=wvsoln_dict['fmax'])
dlamb = (ends[1]-ends[0])/2048.
wcen, dlamb

(7177.4766594754728, 1.597674859144433)

In [30]:
# How is a 2 parameter model?
wv9_true = arutils.func_val(wvsoln_dict['fitc'], inner9/2047., wvsoln_dict['function'], minv=wvsoln_dict['fmin'], maxv=wvsoln_dict['fmax'])
wv9_2 = wcen + dlamb*(inner9-0.5*2047.)
#xdb.xplot(inner9, wv9_true-wv9_2, scatter=True)
# Behaves as expected

## Constant resolution?

In [36]:
wave = arutils.func_val(wvsoln_dict['fitc'], np.arange(2048)/2047., wvsoln_dict['function'], minv=wvsoln_dict['fmin'], maxv=wvsoln_dict['fmax'])
dwv = np.abs(wave-np.roll(wave,1))
dwv[0]=dwv[1]
invR = dwv / wave
#xdb.xplot(invR, scatter=True)
xdb.xplot(dwv, scatter=True)
# Nope

## Examine Fits

In [74]:
# Fitting..
func = 'polynomial'
#func = 'legendre'
#func = 'chebyshev'
mask, param = arutils.robust_polyfit(np.array(wvsoln_dict['xfit']), np.array(wvsoln_dict['yfit']), 3, function=func)
wave = arutils.func_val(param, np.array(wvsoln_dict['xfit']), func)
rms = np.sqrt(np.mean((wave-np.array(wvsoln_dict['yfit']))**2))
param, rms
# All 3 give nearly identical RMS for deg=2 and 3


(array([ 5553.12280678,  3150.81713945,   270.5175083 ,  -149.33064416]),
 0.044789844100892401)

## Cross-correlate

In [75]:
np.array(wvsoln_dict['xfit'])

array([ 0.06885046,  0.07544002,  0.09481151,  0.10056814,  0.10398785,
        0.12362103,  0.13314788,  0.15010683,  0.16384572,  0.17061628,
        0.18512099,  0.19146715,  0.20803243,  0.22320042,  0.23498528,
        0.24408034,  0.25900395,  0.26491111,  0.29682844,  0.30489392,
        0.32505484,  0.34923248,  0.36103746,  0.42556352,  0.43645612,
        0.45674928,  0.46725055,  0.49140797,  0.49952517,  0.52103756,
        0.52945377,  0.56296453,  0.57953858,  0.59460647,  0.60240569,
        0.60877636,  0.62435919,  0.62861852,  0.65668757,  0.70511492,
        0.73334037,  0.75090068,  0.80662985,  0.8192706 ,  0.83947354,
        0.86363829,  0.87298945,  0.90368679,  0.90751297,  0.98585469])